Импорт необходимых библиотек

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import re
import pandas as pd
import numpy as np

Задаем начальные параметры

In [ ]:
url = "http://books.toscrape.com/catalogue/"
ua = UserAgent()
headers = {"User-Agent": ua.chrome}
params = {"page": 1}
all_books = []
session = requests.session()

Парсинг каталога книг

In [ ]:
while True:
    
    # Отпрвляем запрос на страницу и делаем суп
    response = session.get(url+"page-"+str(params["page"])+".html", headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Проверяем, есть ли еще страницы
    if response.status_code != 200:
        break

    # Получаем все книги со страницы
    books = soup.find_all('li', {"class": "col-xs-6 col-sm-4 col-md-3 col-lg-3"})

    # Обрабатываем каждую книгу. На каждой итерации получаем данные о книге и добавляем в основной список книг
    for book in books:
        book_info = {}

        # Получаем название книги со страницы каталога
        book_info["name"] = book.find('h3').getText()

        # Получаем ссылку на страницу с книгой
        book_link = url+book.find('a').get('href')
        
        # Отправляем новый запрос на конкретную книгу и получаем ее описание
        book_response = session.get(book_link, headers=headers)
        book_soup = BeautifulSoup(book_response.content.decode("utf-8"), "html.parser")
        paragraphs = book_soup.find_all('p')
        book_info["description"] = paragraphs[3].getText()

        # Получаем цену книги со странцы каталога
        book_info["price"] = float(book.find('div', {"class": "product_price"}).findChildren()[0].getText()[2:])

        # Получаем количество на складе со странцы книги
        instock =  book_soup.find('p', {"class": "instock availability"}).getText()
        nums = re.findall(r'\d+', instock)
        book_info["in_stock"] = int(nums[0])

        # Добавляем книгу в основной список
        all_books.append(book_info)

    # Для визуализации в выводим на экран информацию об обработанной странице каталога и накопленных книгах
    print(f"Page {params["page"]} passed, {len(all_books)} books were found")
    
    # Увеличиваем значение страницы каталога
    params["page"] += 1


# Обработка всех страниц каталога заняла 13 мин 49 сек 

Из полученных данных cформируем датафрейм и проверим на корректность

In [ ]:
books_data = pd.DataFrame(all_books)

In [ ]:
books_data

Описание некоторых книг отсутствует, вместо него множественный перенос строки. Удалим их и заменим на пустое значение.

In [ ]:
books_data["description"] = books_data["description"].replace({r'.*\n.*': np.nan}, regex=True)

In [ ]:
books_data.iloc[995]

Сохранение данных в csv

In [ ]:
books_data.to_csv("books.csv")